# Intro #
First, I will load in the data from the the notebook on the gender wage gap, because that is what I will be using for this problem set. I will be considering two subsamples of the data. My "young" data will consist of workers with below median experience (experience less than or equal to 10 years) and my "old" data will consist of workers with above median experience (greater than 10 years). It will be interesting to see how the predictive power of our methods compare between groups.

First, we will load in the data.

In [124]:
library(tidyverse)

load('ps1-wage-data')
attach(data)
dim(data)
str(data)

[1] 5150   20

'data.frame':	5150 obs. of  20 variables:
 $ wage : num  9.62 48.08 11.06 13.94 28.85 ...
 $ lwage: num  2.26 3.87 2.4 2.63 3.36 ...
 $ sex  : num  1 0 0 1 1 1 1 0 1 1 ...
 $ shs  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ hsg  : num  0 0 1 0 0 0 1 1 1 0 ...
 $ scl  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ clg  : num  1 1 0 0 1 1 0 0 0 1 ...
 $ ad   : num  0 0 0 1 0 0 0 0 0 0 ...
 $ mw   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ so   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ we   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ ne   : num  1 1 1 1 1 1 1 1 1 1 ...
 $ exp1 : num  7 31 18 25 22 1 42 37 31 4 ...
 $ exp2 : num  0.49 9.61 3.24 6.25 4.84 ...
 $ exp3 : num  0.343 29.791 5.832 15.625 10.648 ...
 $ exp4 : num  0.24 92.35 10.5 39.06 23.43 ...
 $ occ  : Factor w/ 369 levels "10","20","40",..: 159 136 269 23 99 86 226 232 184 146 ...
 $ occ2 : Factor w/ 22 levels "1","2","3","4",..: 11 10 19 1 6 5 17 17 13 10 ...
 $ ind  : Factor w/ 236 levels "370","380","390",..: 204 117 12 165 231 176 171 135 210 201 ...
 $ ind2 : Factor w/ 

Now, we can filter the data and split it into our young and old groups.

In [144]:
youngdata <- filter(data, exp1 <= 10)
young.n <- dim(youngdata)[1]
young.p <- dim(youngdata)[2]

olddata <- filter(data, exp1 > 10)
old.n <- dim(olddata)[1]
old.p <- dim(youngdata)[2]

cat("Number of obs in younger group:", young.n, "\n")
cat("Number of obs in older group:", old.n, "\n")

Number of obs in younger group: 2601 
Number of obs in older group: 2549 


# Problem 1: Sample Splitting #

Sample splitting is a procedure by which we evaluate the out of sample predictive performance of a given model. The basic procedure is this:
1. Randomly split our data into "training" and "testing" groups.
2. Use some method to estimate the parameters of our model using the training data.
3. Use that model to predict our outcome variable for the data in the testing group.
4. Evaluate the performance of the model using the difference between the prediction and the actual outcome variable in the testing group.

Specifically, we will evaluate the performance of our model using out-of-sample $R^2$ and out-of-sample $MSE$.

In general, we want our results to be robust to our choice of training data, so we create multiple different training groups, test the model based on them on our testing group, and average the goodness-of-fit statistics to determine the predictive ability of our model.

We can also use multiple estimation strategies (like OLS, lasso, etc.) and compare those different methods of estimating the same model.

For this sample splitting example, we will use one training and one testing group. However, we could also use multiple training groups and average their $R^2$ and $MSE$, which might yield some more robustness to our data.

First, we randomly split up the data, because we only have 120 observations, I will have 3 groups of training data and 1 group of testing data.

In [149]:
#set seed for replicability
set.seed(1)

#splits into training and testing group
young.random <- sample(1:young.n, size=floor(4/5*young.n))
young.train <- youngdata[young.random,]
young.test <- youngdata[-young.random,]

old.random <- sample(1:old.n, size=floor(4/5*old.n))
old.train <- olddata[old.random,]
old.test <- olddata[-old.random,]

Now we will estimate an additive model and a flexible model for our training group, test them using the testing group, and compare the $R^2_{test}$ and $MSE_{test}$ across the two subsets of data. We will also use lasso to estimate coefficients for the additive model.

In [146]:
#additive model
additive <- lwage ~ (sex + exp1 + occ2)
young.addreg <- lm(additive, data=young.train)
old.addreg <- lm(additive, data=old.train)

p.add <- length(young.addreg$coef)

#flexible
flexible <- lwage ~ sex + (exp1+exp2+exp3+exp4)*(occ2+ind2+mw+so+we)
young.flexreg <- lm(flexible, data=young.train)
old.flexreg <- lm(flexible, data=old.train)

p.flex <- length(young.flexreg$coef)

library(hdm)

#lasso regs
young.lassoreg <- rlasso(additive, data=young.train)
old.lassoreg <- rlasso(additive, data=old.train)

cat("Number of additive regressors", p.add, "\n")
cat("Number of flexible regressors", p.flex, "\n")

Number of additive regressors 24 
Number of flexible regressors 226 


Now we can look at our out-of-sample performance measures for our 3 types of regressions: basic, flexible, and lasso.

To do this, we will predict our outcome variable for our testing data using each of the previously generated models, and find their $R^2$ and $MSE$.

In [147]:
#save testing data wage for comparison
young.actual <- young.test$lwage
old.actual <- old.test$lwage

#young additive
young.trainadd <- predict(young.addreg, newdata=young.test)
young.MSE.add <- sum((young.actual-young.trainadd)^2)/length(young.actual)
young.R2.add <- 1 - young.MSE.add/var(young.actual)

#young flex
young.trainflex <- predict(young.flexreg, newdata=young.test)
young.MSE.flex <- sum((young.actual-young.trainflex)^2)/length(young.actual)
young.R2.flex <- 1 - young.MSE.flex/var(young.actual)

#young lasso
young.trainlasso <- predict(young.lassoreg, newdata=young.test)
young.MSE.lasso <- sum((young.actual-young.trainlasso)^2)/length(young.actual)
young.R2.lasso <- 1 - young.MSE.lasso/var(young.actual)

#old additive
old.trainadd <- predict(old.addreg, newdata=old.test)
old.MSE.add <- sum((old.actual-old.trainadd)^2)/length(old.actual)
old.R2.add <- 1 - old.MSE.add/var(old.actual)

#old flex
old.trainflex <- predict(old.flexreg, newdata=old.test)
old.MSE.flex <- sum((old.actual-old.trainflex)^2)/length(old.actual)
old.R2.flex <- 1 - old.MSE.flex/var(old.actual)

#old lasso
old.trainlasso <- predict(old.lassoreg, newdata=old.test)
old.MSE.lasso <- sum((old.actual-old.trainlasso)^2)/length(old.actual)
old.R2.lasso <- 1 - old.MSE.lasso/var(old.actual)

Warning message in predict.lm(young.flexreg, newdata = young.test):
“prediction from a rank-deficient fit may be misleading”Warning message in predict.lm(old.flexreg, newdata = old.test):
“prediction from a rank-deficient fit may be misleading”

In [148]:
library(xtable)

table <- matrix(0, 3, 4)
table[1,1]   <- young.MSE.add
table[2,1]   <- young.MSE.flex
table[3,1]   <- young.MSE.lasso
table[1,2]   <- young.R2.add
table[2,2]   <- young.R2.flex
table[3,2]   <- young.R2.lasso
table[1,3]   <- old.MSE.add
table[2,3]   <- old.MSE.flex
table[3,3]   <- old.MSE.lasso
table[1,4]   <- old.R2.add
table[2,4]   <- old.R2.flex
table[3,4]   <- old.R2.lasso

rownames(table)<- c("basic reg","flexible reg","lasso reg")
colnames(table)<- c("$MSE_{test, y}$", "$R^2_{test,y}$", "$MSE_{test, o}$", "$R^2_{test,o}$")
tab <- xtable(table, digits=c(3,3,3,3,3))
tab

,"$MSE_{test, y}$","$R^2_{test,y}$","$MSE_{test, o}$","$R^2_{test,o}$"
basic reg,0.2290698,0.1712107,0.3134760,0.15879090
flexible reg,0.3423834,-0.2387653,0.3365846,0.09677906
lasso reg,0.2348890,0.1501564,0.3199382,0.14144949


As we can see, the flexible model does not perform well compared to the additive model, whether or not the additive model is estimated using OLS or lasso. In fact, the flexible regression does so poorly out of sample that its test $R^2$ is negative in the young group. The additive model performs better (but still not extremely way), with little difference between OLS and lasso estimation (OLS is slightly better). The additive model appears to perform slightly better in the younger group, whereas the flexible model performs better in the older group (but still poorly, overall).

# Problem 2: Gender Wage Gap #

In this section we will carry out an analysis of the gap in wage between men and women, among college educated workers. We will utilize the log-linear regression model 

$$\log(Y) = \beta_1D + \beta_2'W + \varepsilon$$ 

where $D$ is an indicator for being female and $W$ is a vector of controls. We can begin by estimating the simpler model 

$$\log(Y) = \beta D+\varepsilon$$ 

This should provide us with the unconditional predictive effect of gender on wages, captured by $\beta$.

We use the same data as the previous problem, but we should first limit the sample to college educated workers.

In [132]:
coldata <- filter(data, clg == 1)
coldata <- select(coldata, -c(shs, hsg, scl, clg, ad))
str(coldata) 

'data.frame':	1636 obs. of  15 variables:
 $ wage : num  9.62 48.08 28.85 11.73 19.23 ...
 $ lwage: num  2.26 3.87 3.36 2.46 2.96 ...
 $ sex  : num  1 0 1 1 1 1 0 0 0 1 ...
 $ mw   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ so   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ we   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ ne   : num  1 1 1 1 1 1 1 1 1 1 ...
 $ exp1 : num  7 31 22 1 4 25 11 8 26 31 ...
 $ exp2 : num  0.49 9.61 4.84 0.01 0.16 6.25 1.21 0.64 6.76 9.61 ...
 $ exp3 : num  0.343 29.791 10.648 0.001 0.064 ...
 $ exp4 : num  0.2401 92.3521 23.4256 0.0001 0.0256 ...
 $ occ  : Factor w/ 369 levels "10","20","40",..: 159 136 99 86 146 7 264 241 267 120 ...
 $ occ2 : Factor w/ 22 levels "1","2","3","4",..: 11 10 6 5 10 1 19 17 19 9 ...
 $ ind  : Factor w/ 236 levels "370","380","390",..: 204 117 231 176 201 190 12 111 152 111 ...
 $ ind2 : Factor w/ 21 levels "2","3","4","5",..: 17 8 21 13 17 16 3 8 10 8 ...


We can now run the simple (unconditional) regression model.

In [133]:
library(sandwich) #used for robust SEs

nocontrol.fit <- lm(lwage ~ sex, data=coldata)
nocontrol.est <- summary(nocontrol.fit)$coef["sex",1]

#robust SEs using vcovHC from sandwich
HCV.coefs <- vcovHC(nocontrol.fit, type='HC')
nocontrol.se <- sqrt(diag(HCV.coefs))[2]

cat("Coefficient:", nocontrol.est, "\n")
cat("Robust SE:", nocontrol.se)

Coefficient: -0.06236138 
Robust SE: 0.02578365

Now we will add controls, which will be the same as the flexible model from earlier (besides education, because of our limited sample).

In [134]:
flex.model <- lwage ~ sex + (exp1+exp2+exp3+exp4)*(occ2+ind2+mw+so+we)

control.fit <- lm(flex.model, data=coldata)
control.est <- control.fit$coef[2]

HCV.coefs <- vcovHC(control.fit, type = 'HC')
control.se <- sqrt(diag(HCV.coefs))[2]

cat("Coefficient with controls:", control.est, "\n")
cat("Robust SE:", control.se)

Coefficient with controls: -0.0462223 
Robust SE: 0.02389685

Notice that while in the original notebook, the controls increased the magnitude of the predictive effect, they decreased the magnitude of the predictive effect in the college educated sample. Now that we have our controlled (conditioned) predictive effect, we can demonstrate the partialling out technique. To do so, we regress our outcome variable and regressor of interest on controls separately. We can then take the residuals and use them in a new model. Put formally, we estimate 

$$\log(Y) = \beta_1' W + \varepsilon \\ D = \beta_2' W + \tilde{d}$$

and store the residuals. We then estimate

$$\varepsilon = \beta \tilde{d} + \xi$$

from the residuals. As it turns out, $\beta$ here is exactly equal to $\beta_1$ from the flexible model above.

In [150]:
#models from which we get resids
flex.y <- lwage ~ (exp1+exp2+exp3+exp4)*(occ2+ind2+mw+so+we)
flex.d <- sex ~ (exp1+exp2+exp3+exp4)*(occ2+ind2+mw+so+we)

#resid storage
resid.Y <- lm(flex.y, data=coldata)$res
resid.D <- lm(flex.d, data=coldata)$res

#fitting from resids
partial.fit <- lm(resid.Y ~ resid.D)
partial.est <- partial.fit$coef[2]

#robust SEs
HCV.coefs <- vcovHC(partial.fit, type = 'HC')
partial.se <- sqrt(diag(HCV.coefs))[2]

cat("Coefficient from partialling out:", partial.est, "\n")
cat("Robust SE:", partial.se, "\n")

#confidence interval
cat("Confidence interval:")
confint(partial.fit)[2,]

Coefficient from partialling out: -0.0462223 
Robust SE: 0.02389685 
Confidence interval:

2.5 %        97.5 % 
-0.0929335874  0.0004889864

Note that both the coefficient and the standard error is exactly the same from the flexible model above.

Partialling out using OLS works very well when the number of regressors, $p$, is low relative to the sample size $n$. Are we in that setting, though? We can check our data dimensions using the flexible model.

In [136]:
n <- dim(coldata)[1]
p <- length(control.fit$coef)

cat("Number of regressors:", p, "\n")
cat("Number of observations", n, "\n")

Number of regressors: 226 
Number of observations 1636 


Perhaps we aren't super concerned with OLS performance in this setting, but $p/n$ is certainly not small here. Therefore, we can use partialling out with lasso regression to improve performance. 

In [137]:
#partialling out the effect of W with Lasso
resid.Y <- rlasso(flex.y, data=coldata)$res
resid.D <- rlasso(flex.d, data=coldata)$res

#fitting from resids
partial.lasso.fit <- lm(resid.Y ~ resid.D)
partial.lasso.est <- partial.lasso.fit$coef[2]

#standard error
HCV.coefs <- vcovHC(partial.lasso.fit, type = 'HC')
partial.lasso.se <- sqrt(diag(HCV.coefs))[2]

cat("Coefficient from lasso partialling:", partial.lasso.est, "\n")
cat("Robust SE:", partial.lasso.se)

Coefficient from lasso partialling: -0.02070625 
Robust SE: 0.02470303

The results here are actually fairly different from those of OLS. The magnitude has decreased by about half, and is not even marginally significant. We can summarize our results in a table.

In [138]:
table<- matrix(0,4,2)
table[1,1]<- nocontrol.est  
table[1,2]<- nocontrol.se   
table[2,1]<- control.est
table[2,2]<- control.se    
table[3,1]<- partial.est  
table[3,2]<- partial.se  
table[4,1]<-  partial.lasso.est
table[4,2]<- partial.lasso.se
colnames(table)<- c("Estimate","Std. Error")
rownames(table)<- c("Without controls", "full reg", "partial reg", "partial reg via lasso")	
tab<- xtable(table, digits=c(3, 3, 4))
tab

,Estimate,Std. Error
Without controls,-0.06236138,0.02578365
full reg,-0.04622230,0.02389685
partial reg,-0.04622230,0.02389685
partial reg via lasso,-0.02070625,0.02470303


Note that although standard errors stay consistent throughout the different models, the magnitude steadily decreases. When doing partial reg via lasso, we see a result that is not statistically significant. This differs from what we see in the unrestricted sample, where adding controls icnreased the estimated predictive effect.

We can now try a VERY flexible model, by including all interaction terms possible given our dataset.

In [139]:
extraflex <- lwage ~ sex + (exp1+exp2+exp3+exp4+occ2+ind2+mw+so+we)^2
control.fit <- lm(extraflex, data=data)

p <- length(control.fit$coef)
cat("Number of regressors:", p, "\n")

Number of regressors: 778 


As we can see, $p/n$ has now exceeded 0.5, so this is falling into a high dimensional case. Therefore, partialling out via lasso is the way we should handle this data. We can compare its estimates to others as well. First, we estimate with standard OLS as a point of comparison, then we use lasso to partial out.

In [140]:
control.est <- control.fit$coef[2]

cat("Coefficient for OLS with extra flex controls", control.est, "\n")


HCV.coefs <- vcovHC(control.fit, type = 'HC');

n= dim(coldata)[1]; p =length(control.fit$coef);

#must adjust standard errors for high dimensional case
control.se <- sqrt(diag(HCV.coefs))[2]*sqrt(n/(n-p)) 

cat("Robust SE:", control.se, "\n")

Coefficient for OLS with extra flex controls -0.07069553 
Robust SE: 0.02207069 


In [141]:
# models
extraflex.y <- lwage ~ (exp1+exp2+exp3+exp4+occ2+ind2+mw+so+we)^2
extraflex.d <- sex ~ (exp1+exp2+exp3+exp4+occ2+ind2+mw+so+we)^2

#partialling out the effect of W with Lasso
resid.Y <- rlasso(extraflex.y, data=coldata)$res
resid.D <- rlasso(extraflex.d, data=coldata)$res

#fitting from resids
partial.lasso.fit <- lm(resid.Y ~ resid.D)
partial.lasso.est <- partial.lasso.fit$coef[2]

#standard error
HCV.coefs <- vcovHC(partial.lasso.fit, type = 'HC')
partial.lasso.se <- sqrt(diag(HCV.coefs))[2]

cat("Coefficient from lasso partialling:", partial.lasso.est, "\n")
cat("Robust SE:", partial.lasso.se)

Coefficient from lasso partialling: -0.01826163 
Robust SE: 0.02487689

In [142]:
table<- matrix(0, 2, 2)
table[1,1]<- control.est
table[1,2]<- control.se    
table[2,1]<-  partial.lasso.est
table[2,2]<- partial.lasso.se 
colnames(table)<- c("Estimate","Std. Error")
rownames(table)<- c("full reg","partial reg via lasso")	
tab<- xtable(table, digits=c(3, 3, 4))
tab

,Estimate,Std. Error
full reg,-0.07069553,0.02207069
partial reg via lasso,-0.01826163,0.02487689


As we can see, the results from our partial regression via lasso differ significantly from the OLS results. The magnitude is very small, and statistically insigificant. In this setting, with $p >> n$, we might trust the results from partialling via lasso more than OLS, because OLS inference does not work properly in the high dimensional case.